In [28]:
import pandas as pd
from tqdm import tqdm
from haversine import haversine
import os

In [8]:
df = pd.read_csv('./Data/20220209_07_21_무당벌레관측.csv')
df_target = pd.read_csv('./absence_지역선정_new.csv')

# 무당벌레 추가 작업 진행

In [9]:
df = df[df.public_positional_accuracy<=1000].reset_index(drop=True)

In [10]:
df = df[(df.Latitude!=0.0)&(df.Longitude!=0.0)].reset_index(drop=True)

In [11]:
#gps 변환
df.Latitude = df.Latitude.astype(float)
df.Longitude = df.Longitude.astype(float)

df_gps = []
for i in tqdm(range(len(df))):
    df_gps.append( (df.Latitude[i], df.Longitude[i]) )
    
    
df['gps'] = df_gps

100%|███████████████████████████████████████████████████████████████████████| 190594/190594 [00:02<00:00, 68326.04it/s]


In [12]:
#gps 변환
df_target.Latitude = df_target.Latitude.astype(float)
df_target.Longitude = df_target.Longitude.astype(float)

df_target_gps = []
for i in tqdm(range(len(df_target))):
    df_target_gps.append( (df_target.Latitude[i], df_target.Longitude[i]) )
    
    
df_target['gps'] = df_target_gps

100%|█████████████████████████████████████████████████████████████████████████████| 721/721 [00:00<00:00, 55609.37it/s]


In [13]:
#sp.로 들어간 애들을 제거

sp_idx = []
for i in range(len(df)):
    if 'sp.' in df.Species[i]:
        sp_idx.append(i)
        

In [14]:
df= df.drop(sp_idx).reset_index(drop=True)

In [15]:
# value_counts 해서 통계적으로 유의미한 30개도 안되는 애들 고르기
under_30 = []
for i in range(len(df.Species.value_counts())):
    if df.Species.value_counts()[i]<30:
        under_30.append(df.Species.value_counts().index[i])

In [16]:
# 고른애들과 이름이 같다면 drop해야하므로 index저장
drop_30 = []
for i in tqdm(range(len(df.Species))):
    for j in under_30:
        if df.Species[i] == j:
            drop_30.append(i)

100%|█████████████████████████████████████████████████████████████████████████| 177580/177580 [05:15<00:00, 562.79it/s]


In [17]:
df= df.drop(drop_30).reset_index(drop=True)

In [18]:
# 속만 있는 경우 제거
one_letter = []
for i in df.Species.value_counts().index:
    if len(i.split(' ')) == 1:
        one_letter.append(i)

In [19]:
drop_one = []
for i in tqdm(range(len(df.Species))):
    for j in one_letter:
        if df.Species[i] == j:
            drop_one.append(i)

100%|███████████████████████████████████████████████████████████████████████| 175834/175834 [00:12<00:00, 14111.00it/s]


In [20]:
df = df.drop(drop_one).reset_index(drop=True)

In [21]:
df = df.drop_duplicates(keep='first')

In [22]:
# 고려되는 전체 종의 개수
len(df.Species.value_counts().index)
#Chilocorus kuwanae 4.5km일때와 달리 1km에서는 30개 미만이라 날아감

90

In [23]:
for i in df.Species.value_counts().index:
    df_target[i+'_18'] = 0

# 주변 종 체크, 관측신뢰도 생성
for i in tqdm(range(len(df_target))): # c9만 뽑아놓은 데이터 프레임
    for j in range(len(df.gps)):  # 전체 무당벌레 데이터 프레임            
        if haversine(df_target.gps[i], df.gps[j], unit='km') <= 18 and df_target.Year[i] == df.Year[j]:
            df_target[df.Species[j]+'_18'][i]+=1

  0%|                                                                                          | 0/721 [00:00<?, ?it/s]<ipython-input-23-c97d816e4e8d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target[df.Species[j]+'_18'][i]+=1
100%|████████████████████████████████████████████████████████████████████████████████| 721/721 [37:12<00:00,  3.10s/it]


In [ ]:
df_target

# 새 추가 작업 진행

In [30]:
# Ebird에서 가져온 새 목록 합치기
os.chdir(path = './Data/E_bird/')

# directory 내의 csv들을 전부 가져와서
for i in range(len(os.listdir())):
    globals()['bird{}'.format(i)] = pd.read_csv(os.listdir()[i],sep='\t')

# for_merge 라는 변수에 모두 합쳐놓음
for_merge = bird1.iloc[0:0]
for i in range(26):
    for_merge = pd.concat([for_merge,globals()['bird{}'.format(i)]])
bird2021 = pd.read_csv("../새2021.csv")
for_merge = pd.concat([for_merge, bird2021]).reset_index(drop=True)


# EBIRD자료는 GPS정확도를 제공하지 않으므로, GPS정확도가 존재한다면 INAT의 자료임
for_merge = for_merge[for_merge.coordinateUncertaintyInMeters.isnull()].reset_index(drop=True)

# 필요한 컬럼들만 남김
for_merge=for_merge[['species','year','month','decimalLatitude','decimalLongitude']].reset_index(drop=True)

# 종 이름이 NAN이면 쓸수 없으므로 DROP
for_merge=for_merge.drop(for_merge[for_merge.species.isnull()==True].index).reset_index(drop=True)

# haversine은 set에 들어있는 gps값을 받음 (lat, lon) 이형식으로 변환하는 과정
for_gps = []
for i in tqdm(range(len(for_merge))):
    for_gps.append( (for_merge['decimalLatitude'][i], for_merge['decimalLongitude'][i]) )
    
for_merge['gps'] = for_gps



for_merge=for_merge[(for_merge.species!='Locustella certhiola')&(for_merge.species!='Locustella ochotensis')&
         (for_merge.species!='Ianthocincla pectoralis')&(for_merge.species!='Luscinia calliope')&
         (for_merge.species!='Iduna aedon')&(for_merge.species!='Phylloscopus sibillatrix')&
         (for_merge.species!='Muscicapa latirostris')&(for_merge.species!='Tiaris bicolor')&
         (for_merge.species!='Tiaris canorus')&(for_merge.species!='Lonchura oryzivora')&
         (for_merge.species!='Lonchura cucullata')&(for_merge.species!='Estrilda coerulescens')&
         (for_merge.species!='alcarius ornatus')]

# 무당벌레는 관측지역에 잡힌 숫자를 믿을 수 없어서 관측 자체만 믿었음, 새 역시도 drop_duplicate로 동일한 절차 진행
for_merge = for_merge.drop_duplicates(keep='first').reset_index(drop=True)


# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in for_merge.species.value_counts().index:
    df_target[i+'_1'] = 0

C:\Users\DAEGYEONGLEE\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\DAEGYEONGLEE\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,16,17,36,37,38,39,41,43,44,46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
100%|████████████████████████████████████████████████████████████████████| 3034469/3034469 [00:26<00:00, 112709.08it/s]


In [31]:
for i in tqdm(range(len(df_target))): # 타겟종만 뽑아놓은 데이터 프레임  
    for j in range(len(for_merge.gps)):  # 전체 새 데이터 프레임
            if haversine(df_target.gps[i], for_merge.gps[j], unit='km') <= 1 and df_target.Year[i] == for_merge.year[j]:
                # 타겟종 관측 좌표와 새 관측좌표의 거리가 1km이하이고, 해당 좌표의 관측 연도가 같다면 count +1
                df_target[for_merge.species[j]+'_1'][i]+=1


  1%|▍                                                                               | 4/721 [01:46<5:17:09, 26.54s/it]<ipython-input-31-600a19410ebe>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target[for_merge.species[j]+'_1'][i]+=1
100%|██████████████████████████████████████████████████████████████████████████████| 721/721 [5:18:03<00:00, 26.47s/it]


In [37]:
df_target.iloc[:,3:]

,Source,Year,Species,Latitude,Longitude,public_positional_accuracy,State,gps,c9_500m,Harmonia axyridis_18,...,Locustella lanceolata_1,Acrocephalus schoenobaenus_1,Acrocephalus dumetorum_1,Cyanerpes cyaneus_1,Luscinia cyane_1,Locustella fluviatilis_1,Phoenicurus phoenicurus_1,Oenanthe pleschanka_1,Taeniopygia guttata_1,Pachyramphus major_1
0,Bugguide.net,2007,Coccinella septempunctata,29.935238,-90.151400,0.0,Louisiana,"(29.935238, -90.1514)",0,1,...,0,0,0,0,0,0,0,0,0,0
1,I-naturalist,2014,Coccinella septempunctata,30.139094,-91.938313,9.0,Louisiana,"(30.139094, -91.938313)",0,2,...,0,0,0,0,0,0,0,0,0,0
2,I-naturalist,2017,Coccinella septempunctata,30.441269,-91.167554,22.0,Louisiana,"(30.4412688656, -91.1675542785)",0,27,...,0,0,0,0,0,0,0,0,0,0
3,I-naturalist,2018,Coccinella septempunctata,30.211919,-92.019677,10.0,Louisiana,"(30.2119190432, -92.0196774882)",0,6,...,0,0,0,0,0,0,0,0,0,0
4,I-naturalist,2018,Coccinella septempunctata,32.439138,-93.663302,0.0,Louisiana,"(32.4391383333, -93.6633016667)",0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,I-naturalist,2019,Harmonia axyridis,44.591483,-104.698455,12.0,Wyoming,"(44.5914833333, -104.698455)",0,1,...,0,0,0,0,0,0,0,0,0,0
717,INAT,2021,Harmonia axyridis,41.045821,-110.876821,65.0,Wyoming,"(41.0458213431, -110.8768207365)",0,1,...,0,0,0,0,0,0,0,0,0,0
718,I-naturalist,2021,Harmonia axyridis,41.870438,-106.558022,5.0,Wyoming,"(41.8704383333, -106.5580216667)",0,1,...,0,0,0,0,0,0,0,0,0,0
719,INAT,2021,Harmonia axyridis,44.799513,-106.963797,4.0,Wyoming,"(44.7995133333, -106.9637966667)",0,1,...,0,0,0,0,0,0,0,0,0,0


In [35]:
df_target.to_csv("../../DOA_집계완료_absence_1km_new.csv")